In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas.io.json import json_normalize
import json
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("./Data"))
# print(os.listdir("../input/2011-data/"))
# print(os.listdir("../input/mag-2011"))

# Any results you write to the current directory are saved as output.

['umass-citation', 'CiteScore_Metrics_2011-2017_Download_25May2018.xlsx', 'ProcessedScopusData.csv', 'PublicationScoreBook.csv', 'CodePublicationMatching.txt', 'umass-citation.tar.gz', 'cora-ie.tar.gz', 'cora-ie']


In [170]:
# JSON_COLUMNS = ["AA","F","J"]
# df = pd.read_csv("../input/mag-2011/mag_2011.csv",converters={column: json.loads for column in JSON_COLUMNS})
df = pd.read_csv("../Data/mag-2011/mag_2011.csv")

In [172]:
df.shape

(1177000, 16)

In [173]:
df.head()

,Unnamed: 0,AA,C,CC,D,ECC,F,Id,J,RId,Ti,VFN,W,Y,kesEntityId,logprob
0,0,"[{'S': 1, 'AuId': 2895256545, 'AfN': 'national...",NaN,38293,2011-04-01,38293,"[{'FId': 2777018310, 'FN': 'caltech 101'}, {'F...",2153635508,"{'JN': 'acm t intel syst tec', 'JId': 2492086750}","[2148603752, 2119821739, 2109943925, 217200036...",libsvm a library for support vector machines,ACM Transactions on Intelligent Systems and Te...,"['library', 'libsvm', 'machines', 'support', '...",2011,95072,-13.274
1,1,"[{'AuId': 581716687, 'S': 1, 'AuN': 'ahmedin j...",NaN,46871,2011-01-01,46871,"[{'FId': 2779372377, 'FN': 'lung cancer suscep...",2613543509,NaN,"[1981989535, 2095761465, 2132528779, 208930075...",global cancer statistics,Ca,"['cancer', 'global', 'statistics']",2011,109169,-13.460
2,2,"[{'S': 1, 'AuId': 2164450181, 'AfN': 'arizona ...",NaN,48290,2011-10-01,48290,"[{'FId': 2778351938, 'FN': 'enterovirus c'}, {...",2132632499,"{'JN': 'mol biol evol', 'JId': 57552105}","[2168696662, 2097706568, 2103546861, 153290820...",mega5 molecular evolutionary genetics analysis...,Molecular Biology and Evolution,"['distance', 'evolutionary', 'genetics', 'like...",2011,113191,-13.508
3,3,"[{'S': 1, 'AuId': 2121939561, 'AfN': 'universi...",NaN,42355,2011-07-06,42355,"[{'FId': 87146676, 'FN': 'gsp algorithm'}, {'F...",2140190241,NaN,"[2339804494, 2156909104, 2432517183, 210286575...",data mining concepts and techniques,NaN,"['concepts', 'data', 'mining', 'techniques']",2011,114591,-13.524
4,4,"[{'AuId': 2300357236, 'S': 1, 'AuN': 'laurence...",NaN,13533,2011-01-01,13533,NaN,2204447578,NaN,NaN,analise de conteudo,NaN,"['analise', 'conteudo']",2011,158426,-13.924


In [174]:
df.isna().sum()

Unnamed: 0           0
AA                   0
C              1115805
CC                   0
D                    0
ECC                  0
F                 6085
Id                   0
J               374096
RId             186434
Ti                   0
VFN             276961
W                   12
Y                    0
kesEntityId          0
logprob              0
dtype: int64

In [175]:
def preprocess(df):
    df = df.drop(['C','J'],axis = 1)
    df = df.dropna(subset=['RId','F','VFN','W'])
    return df

In [176]:
df = preprocess(df)

In [177]:
df.isna().sum()

Unnamed: 0     0
AA             0
CC             0
D              0
ECC            0
F              0
Id             0
RId            0
Ti             0
VFN            0
W              0
Y              0
kesEntityId    0
logprob        0
dtype: int64

In [157]:
df.head()

,Unnamed: 0,AA,CC,D,ECC,F,Id,RId,Ti,VFN,W,Y,kesEntityId,logprob
0,0,"[{'S': 1, 'AuId': 2895256545, 'AfN': 'national...",38293,2011-04-01,38293,"[{'FId': 2777018310, 'FN': 'caltech 101'}, {'F...",2153635508,"[2148603752, 2119821739, 2109943925, 217200036...",libsvm a library for support vector machines,ACM Transactions on Intelligent Systems and Te...,"['library', 'libsvm', 'machines', 'support', '...",2011,95072,-13.274
1,1,"[{'AuId': 581716687, 'S': 1, 'AuN': 'ahmedin j...",46871,2011-01-01,46871,"[{'FId': 2779372377, 'FN': 'lung cancer suscep...",2613543509,"[1981989535, 2095761465, 2132528779, 208930075...",global cancer statistics,Ca,"['cancer', 'global', 'statistics']",2011,109169,-13.460
2,2,"[{'S': 1, 'AuId': 2164450181, 'AfN': 'arizona ...",48290,2011-10-01,48290,"[{'FId': 2778351938, 'FN': 'enterovirus c'}, {...",2132632499,"[2168696662, 2097706568, 2103546861, 153290820...",mega5 molecular evolutionary genetics analysis...,Molecular Biology and Evolution,"['distance', 'evolutionary', 'genetics', 'like...",2011,113191,-13.508
6,6,"[{'S': 1, 'AuId': 2120435251, 'AfN': 'isrec', ...",32471,2011-03-01,32471,"[{'FId': 2776199265, 'FN': 'the hallmarks of c...",2117692326,"[2034269086, 2097259163, 2023427658, 196408120...",hallmarks of cancer the next generation,Cell,"['cancer', 'generation', 'hallmarks', 'next']",2011,193981,-14.161
7,7,"[{'S': 1, 'AuId': 317192602, 'AfN': 'french in...",12801,2011-02-01,12801,"[{'FId': 24138899, 'FN': 'instance based learn...",2101234009,"[2153635508, 2118585731, 2063978378, 209736028...",scikit learn machine learning in python,Journal of Machine Learning Research,"['learn', 'learning', 'machine', 'python', 'sc...",2011,264962,-14.513


In [178]:
df_pub = pd.read_csv("../input/scopus-data/ProcessedScopusData.csv")

In [179]:
df_pub.head()

,Unnamed: 0,Scopus SourceID,Title,CiteScore,Percentile,Citation Count,Scholarly Output,Percent Cited,SNIP,SJR,RANK,Rank Out Of,Publisher,Type,OpenAccess,Scopus ASJC Code (Sub-subject Area),Scopus Sub-Subject Area,Quartile,Top 10% (CiteScore Percentile),Scopus SourceID.1,Print-ISSN,E-ISSN,Subject Area
0,0,28773.0,Ca-A Cancer Journal for Clinicians,130.47,99.0,16961.0,130.0,70.0,88.164,61.786,1.0,120.0,Wiley-Blackwell,Journal,NO,2720.0,Hematology,Quartile 1,Top 10%,https://www.scopus.com/sourceid/28773,00079235,15424863,Medicine
1,1,28773.0,Ca-A Cancer Journal for Clinicians,130.47,99.0,16961.0,130.0,70.0,88.164,61.786,1.0,323.0,Wiley-Blackwell,Journal,NO,2730.0,Oncology,Quartile 1,Top 10%,https://www.scopus.com/sourceid/28773,00079235,15424863,Medicine
2,2,19434.0,MMWR. Recommendations and reports : Morbidity ...,63.12,99.0,1010.0,16.0,100.0,32.534,34.638,1.0,87.0,Centers for Disease Control and Prevention (CDC),Journal,YES,2713.0,Epidemiology,Quartile 1,Top 10%,https://www.scopus.com/sourceid/19434,10575987,15458601,Medicine
3,3,19434.0,MMWR. Recommendations and reports : Morbidity ...,63.12,99.0,1010.0,16.0,100.0,32.534,34.638,1.0,241.0,Centers for Disease Control and Prevention (CDC),Journal,YES,3306.0,Health(social science),Quartile 1,Top 10%,https://www.scopus.com/sourceid/19434,10575987,15458601,Social Sciences
4,4,19434.0,MMWR. Recommendations and reports : Morbidity ...,63.12,99.0,1010.0,16.0,100.0,32.534,34.638,1.0,106.0,Centers for Disease Control and Prevention (CDC),Journal,YES,2307.0,"Health, Toxicology and Mutagenesis",Quartile 1,Top 10%,https://www.scopus.com/sourceid/19434,10575987,15458601,Environmental Science


In [207]:
def append_pub_score(df):
    print("Unique Publications from Scopus:",len(df_pub.Title.unique()))
    # Analysing Common Publications
    vfn_list = df.VFN.unique().tolist()
    scopus_list = df_pub.Title.unique().tolist()
    count = 0
    for pub in scopus_list:
        if pub in vfn_list:
            count +=1
    print("Publications present in Dataset:",count)
    print("Shape Before",df.shape)
    df = df[df.VFN.isin(scopus_list)]
    print("Shape After",df.shape)
    #score appending
    score_dict = pd.Series(df_pub.SJR.values,index=df_pub.Title).to_dict()
    df["Publication_Rank"] = df.apply(lambda row: score_dict[row["VFN"]],axis = 1)
    return df

def extract_field(row):
    val = row["F"]
    index = val.rfind("FN")
    val = val[index+3:len(val)]
    val = val.strip(":}] '")
    return val

In [186]:
df = append_pub_score(df)

Unique Publications from Scopus: 1231
Publications present in Dataset: 883
Shape Before (105906, 14)
Shape After (105906, 14)


In [209]:
df["Subject Field"] = df.apply(lambda row:extract_field(row),axis=1)

In [210]:
df.head()

,Unnamed: 0,AA,CC,D,ECC,F,Id,RId,Ti,VFN,W,Y,kesEntityId,logprob,Publication_Rank,Subject Field
0,0,"[{'S': 1, 'AuId': 2895256545, 'AfN': 'national...",38293,2011-04-01,38293,"[{'FId': 2777018310, 'FN': 'caltech 101'}, {'F...",2153635508,"[2148603752, 2119821739, 2109943925, 217200036...",libsvm a library for support vector machines,ACM Transactions on Intelligent Systems and Te...,"['library', 'libsvm', 'machines', 'support', '...",2011,95072,-13.274,0.874,computer science
2,2,"[{'S': 1, 'AuId': 2164450181, 'AfN': 'arizona ...",48290,2011-10-01,48290,"[{'FId': 2778351938, 'FN': 'enterovirus c'}, {...",2132632499,"[2168696662, 2097706568, 2103546861, 153290820...",mega5 molecular evolutionary genetics analysis...,Molecular Biology and Evolution,"['distance', 'evolutionary', 'genetics', 'like...",2011,113191,-13.508,5.475,biology
6,6,"[{'S': 1, 'AuId': 2120435251, 'AfN': 'isrec', ...",32471,2011-03-01,32471,"[{'FId': 2776199265, 'FN': 'the hallmarks of c...",2117692326,"[2034269086, 2097259163, 2023427658, 196408120...",hallmarks of cancer the next generation,Cell,"['cancer', 'generation', 'hallmarks', 'next']",2011,193981,-14.161,25.137,medicine
7,7,"[{'S': 1, 'AuId': 317192602, 'AfN': 'french in...",12801,2011-02-01,12801,"[{'FId': 24138899, 'FN': 'instance based learn...",2101234009,"[2153635508, 2118585731, 2063978378, 209736028...",scikit learn machine learning in python,Journal of Machine Learning Research,"['learn', 'learning', 'machine', 'python', 'sc...",2011,264962,-14.513,1.271,computer science
28,28,"[{'S': 1, 'AuId': 281745206, 'AfN': 'universit...",7383,2011-03-01,7383,"[{'FId': 2776282453, 'FN': 'milan criteria'}, ...",2163403599,"[1971837077, 1994193851, 2057378223, 213032537...",management of hepatocellular carcinoma an update,Hepatology,"['carcinoma', 'hepatocellular', 'management', ...",2011,489309,-15.158,5.541,medicine


In [212]:
df["Subject Field"].unique()

array(['computer science', 'biology', 'medicine', 'mathematics',
       'psychology', 'materials science', 'geology', 'physics',
       'chemistry', 'environmental science', 'economics',
       'distributed computing', 'nanotechnology', 'business', 'geography',
       'engineering', 'optics', 'political science', 'applied psychology',
       'sociology', 'ecology', 'mineralogy', 'expansive', 'agronomy',
       'creatures', 'composite material', 'social psychology',
       'geomorphology', 'history', 'neuroscience', 'environmental ethics',
       'elaboration', 'socioeconomics', 'environmental engineering',
       'crystallography', 'innovator', 'oceanography',
       'spatial configuration', 'transferability',
       'environmental protection', 'management science',
       'atmospheric sciences', 'analytical chemistry', 'lanperisone',
       'twenty first century', 'physiology', 'scientific study',
       'philosophy', 'climatology', 'physical chemistry',
       'theoretical computer s

In [213]:
df.shape

(105906, 16)